In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from pathlib import Path

In [ ]:
jvs_path = Path("/Users/takeshitashunji/Downloads/jvs_ver1 2")

# jvs001~jvs100
jvs_list = [f"{i:03}" for i in range(1, 101)]

# labファイルのパスを取得
lab_paths = {
    i: jvs_path / f"jvs{i}" / f"parallel100/lab/mon" / f"VOICEACTRESS100_001.lab"
    for i in jvs_list
    if i not in ["006", "028"]
}
print(len(lab_paths))
print(lab_paths["001"])

In [ ]:
df_lab = []
for jvs_num, lab_path in lab_paths.items():
    with open(lab_path) as f:
        for phoneme_idx, line in enumerate(f):
            start, end, phoneme = line.split()
            df_lab.append(
                {
                    "jvs_num": jvs_num,
                    "start": float(start),
                    "end": float(end),
                    "phoneme": phoneme,
                    "phoneme_idx": phoneme_idx,
                }
            )

df_lab = pd.DataFrame(df_lab)
df_lab["duration"] = df_lab["end"] - df_lab["start"]
df_lab.head()

In [ ]:
df_lab[df_lab["jvs_num"] == "001"]["phoneme"].values

In [ ]:
duration_arr = []
for jvs_num in df_lab["jvs_num"].unique():
    df_jvs = df_lab[df_lab["jvs_num"] == jvs_num]
    duration_arr.append(df_jvs["duration"].values)

duration_arr = np.array(duration_arr)
print(duration_arr.shape)

# 各音素分散のヒストグラム
duration_center = np.mean(duration_arr, axis=0)
duration_arr_center = duration_arr - duration_center
duration_var = np.var(duration_arr_center, axis=0)
print(duration_var.shape)
plt.figure(figsize=(20, 10))
x = df_lab[df_lab["jvs_num"] == "001"]["phoneme"].values
plt.plot(range(len(x)), duration_var)
plt.xticks(range(len(x)), x)
# plt.ylim(0, 0.01)
plt.show()

In [ ]:
duration_arr.T.shape

In [ ]:
df_lab[df_lab["jvs_num"] == "001"][["phoneme", "phoneme_idx"]].values[50:]

In [ ]:
duration_arr = []
for jvs_num in df_lab["jvs_num"].unique():
    df_jvs = df_lab[df_lab["jvs_num"] == jvs_num]
    duration_arr.append(df_jvs["duration"].values)

duration_arr = np.array(duration_arr)
print(duration_arr.shape)

plt.figure(figsize=(20, 10))
x = df_lab[df_lab["jvs_num"] == "001"]["phoneme"].values
plt.boxplot(duration_arr, labels=x)
# plt.xticks(range(len(x)), x)
# plt.ylim(0, 0.01)
plt.show()

In [ ]:
df_lab[df_lab["phoneme_idx"] == 61][df_lab["duration"] > 0.4]

In [ ]:
df_lab[df_lab["phoneme_idx"] == 61][df_lab["duration"] < 0.05]

In [ ]:
for i, p in enumerate(df_lab[df_lab["jvs_num"] == "001"]["phoneme"].values):
    print(i, p)

In [ ]:
# 各音素の最大最小の人を表示

duration_max = np.argmax(duration_arr, axis=0)
duration_min = np.argmin(duration_arr, axis=0)

# t
phoneme_num = 11
print(duration_max[phoneme_num], duration_min[phoneme_num])
print(jvs_list[duration_max[phoneme_num]], jvs_list[duration_min[phoneme_num]])

In [ ]:
# 各音素(順番を保持)のdurationの分布
sns.barplot(x="phoneme", y="duration", data=df_lab, ci=None)

In [ ]:
# 各音素のdurationの分布, 分散も出しておく

sns.barplot(
    y="phoneme", x="duration", data=df_lab.groupby("phoneme").mean().reset_index()
)

In [ ]:
sns.barplot(
    y="phoneme",
    x="duration",
    data=df_lab.groupby("phoneme").agg({"duration": np.var}).reset_index(),
)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# データをグループ化して平均値を計算し、データフレームを取得する
df_mean = df_lab.groupby("phoneme").mean().reset_index()

# バーの散らばり具合を計算する
df_std = df_lab.groupby("phoneme").std().reset_index()

# グラフの描画
fig, ax = plt.subplots()
sns.barplot(x="duration", y="phoneme", data=df_mean, ax=ax)  # 平均値のバーを描画
sns.scatterplot(
    x="duration", y="phoneme", data=df_std, color="red", ax=ax
)  # 散らばり具合のプロットを描画

# グラフの設定
ax.set_xlabel("Duration")
ax.set_ylabel("Phoneme")
ax.set_title("Bar Plot with Dispersion")

# グラフの表示
plt.show()